In [ ]:
"""
    Ranking:
        1.1 Hierarchical Clustering
        1.2 Map labels to future matches as home and away of past season
            1.2.1 If team is new, label it as normal (which is a not promoted team)
        1.3 Compute difference in ranking (NO because of no sequential)
        1.4 Future work: study the week development of the league to see if there is a point where
            It can be started to predict the actual behavior of the team as the promotion label
    Prediction:
        2.0 Drop features --> computation differences
        2.1 Features TEST
            2.1.0 First match as median of past season (already started) TODO
            2.1.2 Median of current season
            2.1.2.1 Average at home/away
            2.1.2.2 Past average of last n games (test with 3, 5, etc)
            2.1.2.3 Average of last n games at home/away
"""